# Test loop closure

In [ ]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go
from copy import deepcopy

import planeslam.io as io
from planeslam.scan import pc_to_scan
from planeslam.general import NED_to_ENU, trajectory_plot_trace
from planeslam.geometry.util import quat_to_R

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(suppress=True)

### Load AirSim data

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'lidar', 'Drone0')
PCs = io.read_lidar_bin(binpath)

# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [ ]:
# Subsample data
sub_factor = 5
PCs = PCs[::sub_factor]
drone_positions = drone_positions[::sub_factor]
drone_orientations = drone_orientations[::sub_factor]

In [ ]:
# Convert to ENU
num_scans = len(PCs)

for i in range(num_scans):
    PCs[i] = NED_to_ENU(PCs[i])

drone_positions = NED_to_ENU(drone_positions)
drone_orientations = NED_to_ENU(drone_orientations)

drone_rotations = np.zeros((3,3,num_scans))
for i in range(num_scans):
    drone_rotations[:,:,i] = quat_to_R(drone_orientations[i])

In [ ]:
# Plot ground-truth trajectory
gt_traj_trace = go.Scatter3d(x=drone_positions[:,0], y=drone_positions[:,1], z=drone_positions[:,2], 
    marker=dict(size=5), hovertext=np.arange(len(drone_positions)))
fig = go.Figure(data=gt_traj_trace)
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
from planeslam.registration import robust_GN_register

init_pose = (quat_to_R(drone_orientations[0]), drone_positions[0,:].copy())

# For airsim
N = len(PCs)

# Relative transformations
R_hats = []
t_hats = []

# Absolute poses
R_abs, t_abs = init_pose
poses = N * [None]
poses[0] = (R_abs, t_abs)

# Scans
scans = N * [None]
scans[0] = pc_to_scan(PCs[0])

# Pose graph

for i in range(1, N):
    P = PCs[i]
    
    # Extract scan
    scans[i] = pc_to_scan(P)
    scans[i].remove_small_planes(area_thresh=5.0)

    # Registration
    R_hat, t_hat = robust_GN_register(scans[i], scans[i-1])
    t_abs += (R_abs @ t_hat).flatten()
    R_abs = R_hat @ R_abs
    poses[i] = (R_abs, t_abs)

In [ ]:
poses

In [ ]:
gt_traj_trace = go.Scatter3d(x=drone_positions[:,0], y=drone_positions[:,1], z=drone_positions[:,2], 
    marker=dict(size=5), hovertext=np.arange(len(drone_positions)), name="Ground-truth")
est_traj_trace = go.Scatter3d(x=traj_est[:,0], y=traj_est[:,1], z=traj_est[:,2], 
    marker=dict(size=5), hovertext=np.arange(len(traj_est)), name="Estimated")
fig = go.Figure(data=[gt_traj_trace, est_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'), legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))
fig.show()